In this notebook, we'll start to construct our dataset and explore a few modeling options so that later, we can create an optimized version of our code for production

# Dataset and feature construction

## Step 1: Get the high-level structure of our dataset in place